## 3. Nachvollziehen der Beispiele aus der Vorlesung
Das Netz aus der Vorlesung verwendet als Aktivierungsfunktion den Tangens hyperbolicus
(`np.tanh()`). Passen Sie die Funktionen `sigmoid()` und `sigmoid_prime()` entsprechend
an. 

**Achtung**: kommentieren Sie den bisherigen Code für die Sigmoidfunktion nur aus, wir
werden ihn in der nächsten Aufgabe nochmals benötigen. Da die Ausgangswerte von $tanh$
im Intervall [−1, 1] statt [0, 1] liegen, müssen wir hierfür nochmals die Funktion `evaluate()` entsprechend anpassen.

Vollziehen Sie die 3 Beispiele aus der Vorlesung nach.

Das folgende Beispiel bezieht sich auf das letzte Beispiel aus der Vorlesung welches unten abgebildet ist oder [hier](https://playground.tensorflow.org./#activation=tanh&batchSize=10&dataset=xor&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=3,2&seed=0.55295&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false) ausführbar ist.
![Vorlesungsbeispiel 3](./data/experiment_3.png)

Das Netzwerk kann durch Anpassung der `layers`-Variable auch an die vorherigen Beispiele angepasst werden. 

In [ ]:
# Netzwerkparameter laut Aufgabenbeschreibung

mbs = 10                    # Größe des Mini-Batches
eta = 0.03                  # Lernrate eta
epochs = 150                # Anzahl der Epochen

# Anzahl der Neuronen pro Schicht (2 Eingabe, 2 versteckte Schichten mit 2 Neuronen, 1 Ausgabe)
layers = [2, 3, 2, 1]

num_layers = len(layers)    # Anzahl der Schichten im Netzwerk
num_layers

In [ ]:
import numpy as np
biases = [np.random.randn(y, 1) for y in layers[1:]]                        # Schwellwerte
weights = [np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])]  # Gewichte

### Datensätze generieren

In [ ]:
import numpy as np

from auxiliary import get_labels, build_design_matrix

# Erstellen der Datenpunkte im vorgegebenen Bereich für Training und Test
X_train = np.random.uniform(-6, 6, (200, 2))
X_test = np.random.uniform(-6, 6, (200, 2))

y_train = get_labels(X_train)
y_test = get_labels(X_test)

w_x = np.array([1, 0])
w_y = np.array([0, 1])

xv = np.linspace(-6, 6, 100)
yv = np.linspace(-6, 6, 100)

X, Y = np.meshgrid(xv, yv)

design = np.c_[X.ravel(), Y.ravel()]

design_X_train  = build_design_matrix(X_train, w_x, w_y)
design_X_test   = build_design_matrix(X_test, w_x, w_y)

### Angepasste Hilfsfunktionen

Die Aktivierungsfunktionen werden im nächsten Schritt wie in der Aufgabenbeschreibung vorgegeben angepasst.

In [ ]:
# Sigmoid (vektorisiert)
def sigmoid(z):
    """~The sigmoid function.~ The tangent hyperbolic function."""
    #return 1.0/(1.0+np.exp(-z))
    return np.tanh(z)

# Ableitung des Sigmoids
def sigmoid_prime(z):
    """~Derivative of the sigmoid function.~ Derivative of the tangent hyperbolic function."""
    #return sigmoid(z)*(1-sigmoid(z))
    return 1.0 - np.tanh(z)**2

# Ableitung der MSE-Kostenfunktion
def cost_derivative(output_activations, y):
    """Return the vector of partial derivatives \partial C_x /
    \partial a for the output activations."""
    return (output_activations-y)

# MSE-Kostenfunktion
def cost(output_activations, y):
    """
    Return the MSE cost.
    #### Arguments:
        output_activations -- The output activations from the network
        y -- The true labels
    #### Returns:
        The MSE cost as a float value
    """
    return (0.5 * np.linalg.norm(output_activations - y) ** 2)

### Feedforward inkludieren

In [ ]:
def feedforward(a):
    """Return the output of the network if ``a`` is input."""
    for b, w in zip(biases, weights):
        a = sigmoid(np.dot(w, a)+b)
    return a

### Evaluierungsfunktion anpassen

Wie beschrieben muss die Evaluierungsfunktion angepasst werden, damit sie mit den Ausgabewerten von $tanh$ funktioniert. Da $tanh$ Werte im Intervall [−1, 1] ausgibt, können wir als Klassengrenze 0 verwenden. Werte größer 0 werden der Klasse 1 zugeordnet, Werte kleiner oder gleich 0 der Klasse 0. 

Diese Änderung erfolgt in **Codezeile 23**

In [ ]:
def evaluate(x2, y2):
    """Return the number of test inputs for which the neural
    network outputs the correct result. Note that the neural
    network's output is assumed to be the index of whichever
    neuron in the final layer has the highest activation."""
    
    correct = 0 # Anzahl korrekt klassifizierter Testbeispiele
    loss = []   # Liste zur Speicherung der Verluste (Erweiterung laut Aufgabenbeschreibung)
    
    # Gehe den Testdatensatz durch
    for i in range(0, x2.shape[0]):
        x = np.reshape(x2[i,:],(x2.shape[1],1)).copy()
        if len(y2.shape) == 2:
            y = np.reshape(y2[i,:],(y2.shape[1],1)).copy()
        else:
            y = y2[i].copy()
        
        # Vorwärtslauf
        ypred = feedforward(x)
        
        # Die Vorhersage ist 1, wenn die Ausgabe > 0 ist, sonst 0
        # ÄNDERUNG: Da tanh (-1 bis 1) als Aktivierungsfunktion verwendet wird, ist die Grenze jetzt bei 0
        cpred = 1 if ypred >= 0 else 0

        c = y
        
        # Falls beide übereinstimmen, addiere zur Gesamtzahl
        if c == cpred:
            correct += 1

        # Loss berechnen und speichern (Erweiterung laut Aufgabenbeschreibung)
        loss.append(cost(ypred, y))
        
    return correct, loss

### Backpropagation-Algorithmus

In [ ]:
def backprop(x, y):
    """Return a tuple ``(\nabla_b, nabla_w)`` representing the
    gradient for the cost function C_x.  ``nabla_b`` and
    ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
    to ``self.biases`` and ``self.weights``."""
    
    # Initialisiere Updates für Schwellwerte und Gewichte
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]
    
    # Vorwärtslauf
    activation = x # Initialisierung a^1 = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    for b, w in zip(biases, weights):
        z = np.dot(w, activation) + b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
    
    # Rückwärtslauf
    delta = cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) # Fehler am Output
    nabla_b[-1] = delta # Update Schwellwert in der Ausgangsschicht
    nabla_w[-1] = np.dot(delta, activations[-2].transpose()) # Update Gewichte in der Ausgangsschicht
    for l in range(2, num_layers): # Backpropagation
        z = zs[-l] # gewichteter Input
        sp = sigmoid_prime(z) # Ableitung der Aktivierungsfunktion
        delta = np.dot(weights[-l+1].transpose(), delta) * sp # Fehler in Schicht l
        nabla_b[-l] = delta # Update Schwellwert 
        nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) # Update Gewichte

    return (nabla_b, nabla_w)

### Minibatches

In [ ]:
def update_mini_batch(xmb, ymb, eta):
    """Update the network's weights and biases by applying
    gradient descent using backpropagation to a single mini batch.
    The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
    is the learning rate."""
    global weights
    global biases

    # Initialisiere Updates für Schwellwerte und Gewichte
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]
    
    # Gehe durch alle Beispielpaare im Minibatch
    for i in range(xmb.shape[0]):
        x = np.reshape(xmb[i,:],(xmb.shape[1],1)).copy()
        if len(ymb.shape) == 2:
            y = np.reshape(ymb[i,:],(ymb.shape[1],1)).copy()
        else:
            y = ymb[i].copy()
        
        # Berechne Updates für alle Schichten über Backprop
        delta_nabla_b, delta_nabla_w = backprop(x, y)
        
        # Addiere einzelne Updates auf
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    
    # Berechne neue Gewichte
    weights = [w-(eta/xmb.shape[0])*nw
                    for w, nw in zip(weights, nabla_w)]
    biases = [b-(eta/xmb.shape[0])*nb
                   for b, nb in zip(biases, nabla_b)]
    
    return (weights, biases)

### Stochastischer Gradientenabstieg (SGD)

In [ ]:
def SGD(x0, y0, epochs, mini_batch_size, eta, x2, y2):

    n_test = x2.shape[0] # Anzahl Testdaten
    n = x0.shape[0]      # Anzahl Trainingsdaten
    
    precision_curve = []  # Liste zur Speicherung der Präzisionswerte (Erweiterung laut Aufgabenbeschreibung)
    mse_curve = []        # Liste zur Speicherung der MSE-Werte (Erweiterung laut Aufgabenbeschreibung)

    # gehe durch alle Epochen
    acc_val = np.zeros(epochs)
    print("| Epochs | Precision | Loss   |")
    for j in range(epochs):
        
        # Bringe die Trainingsdaten in eine zufällige Reihenfolge für jede Epoche
        p = np.random.permutation(n) # Zufällige Permutation aller Indizes von 0 .. n-1
        x0 = x0[p,:]
        y0 = y0[p]
        
        # Zerlege den permutierten Datensatz in Minibatches 
        for k in range(0, n, mini_batch_size):
            xmb = x0[k:k+mini_batch_size,:]
            if len(y0.shape) == 2:
                ymb = y0[k:k+mini_batch_size,:]
            else:
                ymb = y0[k:k+mini_batch_size]
            update_mini_batch(xmb, ymb, eta)
        
        # Gib Performance aus
        acc_val[j], loss = evaluate(x2, y2)

        precision_curve.append(acc_val[j] / n_test)  # Präzisionswert speichern
        mse_curve.append(np.mean(loss))              # MSE-Wert speichern

        if j % 10 == 0 or j == epochs - 1:
            print("|  {:>5} |   {:>6.4f}  | {:>6.4f} |".format(j+1, precision_curve[-1], mse_curve[-1]))
    
    return acc_val, precision_curve, mse_curve

### Training des Netzwerk

In [ ]:
acc_val, precision, loss = SGD(design_X_train, y_train, epochs, mbs, eta, X_test, y_test)

In [ ]:
print("Shape of precision array:", np.array(precision).shape)
print("Shape of loss array:", np.array(loss).shape)

### Darstellung der Lernkurven

In [ ]:
from matplotlib import pyplot as plt

### Darstellen der Lernkurven
# Genauigkeit-Kurve
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(epochs), precision, label='Genauigkeit', color='blue')
plt.xlabel('Epochen')
plt.ylabel('Genauigkeit')
plt.title('Genauigkeit-Kurve')
plt.grid()
# MSE-Kurve
plt.subplot(1, 2, 2)
plt.plot(range(epochs), loss, label='MSE', color='red')
plt.xlabel('Epochen')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('MSE-Kurve')
plt.grid()
plt.tight_layout()
plt.show()

### Entscheidungsgrenzen visualisieren

In [ ]:
mlp_design_decision = feedforward(design.T)

figure, axis = plt.subplots(1, 1, figsize=(6, 6), dpi=80)
axis_limits = [-6, 6]

# Plot der Entscheidungsfunktion
axis.pcolor(X, Y, mlp_design_decision.reshape(100,100), alpha=.8)
axis.grid(True)
axis.set_title("Entscheidungsfunktion 100 x 100 Gitter und Trainingsdaten")
axis.set_xlabel("x")
axis.set_ylabel("y")
axis.axhline(0, color='black', linewidth=0.5)
axis.axvline(0, color='black', linewidth=0.5)
axis.set_xlim(axis_limits)
axis.set_ylim(axis_limits)
axis.set_aspect('equal')

# Scatter-Plot der Labels
plt.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], color='blue', label='Label 1')
plt.scatter(X_train[y_train == 0, 0], X_train[y_train == 0, 1], color='red', label='Label 0')

plt.show()